In [6]:
!unzip data.zip
%ls

Archive:  data.zip
   creating: data/
   creating: data/normal/
  inflating: data/normal/1.jpg       
  inflating: data/normal/10.jpg      
  inflating: data/normal/100.jpg     
  inflating: data/normal/11.jpg      
  inflating: data/normal/12.jpg      
  inflating: data/normal/13.jpg      
  inflating: data/normal/14.jpg      
  inflating: data/normal/15.jpg      
  inflating: data/normal/16.jpg      
  inflating: data/normal/17.jpg      
  inflating: data/normal/18.jpg      
  inflating: data/normal/19.jpg      
  inflating: data/normal/2.jpg       
  inflating: data/normal/20.jpg      
  inflating: data/normal/21.jpg      
  inflating: data/normal/22.jpg      
  inflating: data/normal/23.jpg      
  inflating: data/normal/24.jpg      
  inflating: data/normal/25.jpg      
  inflating: data/normal/26.jpg      
  inflating: data/normal/27.jpg      
  inflating: data/normal/28.jpg      
  inflating: data/normal/29.jpg      
  inflating: data/normal/3.jpg       
  inflating: data/normal

# clip

In [7]:
import torch
import os
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

In [8]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_dir, transform=None):
        self.dataset_dir = dataset_dir
        self.transform = transform
        self.images, self.labels = [], []
        for dirpath, _, filenames in os.walk(dataset_dir):
            for filename in filenames:
                label = 0 if 'normal' in dirpath else 1
                img_path = os.path.join(dirpath, filename)
                self.images.append(img_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx], self.images[idx]

In [ ]:
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
def classify_image(image_path):
    image = Image.open(image_path).convert("RGB")
    texts = [
        "Обычное изображение",
        "Фишинг: фальшивый QR-код",
        "Фишинг: голосование",
        "Фишинг: поддельная страница входа в соцсети",
        "Фишинг: предложение банковских услуг или биржи",
    ] # < 77 символов

    inputs = clip_processor(text=texts, images=image, return_tensors="pt", padding=True)

    with torch.no_grad():
        outputs = clip_model(**inputs)
        logits_per_image = outputs.logits_per_image.softmax(dim=1)

    phishing_prob = logits_per_image[0, 1].item() * 10
    return phishing_prob

In [ ]:
dataset_path = "/content/data"
dataset = ImageDataset(dataset_path)

In [ ]:
results = {}
for _, _, image_path in dataset:
    confidence = classify_image(image_path)
    results[image_path] = confidence
    print(f"{image_path}: Уверенность фишинга {confidence:.1f} из 10")

/content/data/phishing/34.jpg: Уверенность фишинга 2.5 из 10
/content/data/phishing/46.jpg: Уверенность фишинга 3.4 из 10
/content/data/phishing/83.jpg: Уверенность фишинга 7.7 из 10
/content/data/phishing/51.jpg: Уверенность фишинга 4.7 из 10
/content/data/phishing/78.jpg: Уверенность фишинга 0.7 из 10
/content/data/phishing/77.jpg: Уверенность фишинга 0.4 из 10
/content/data/phishing/000093.jpg: Уверенность фишинга 1.6 из 10
/content/data/phishing/96.jpg: Уверенность фишинга 8.0 из 10
/content/data/phishing/28.jpg: Уверенность фишинга 3.5 из 10
/content/data/phishing/57.jpg: Уверенность фишинга 0.8 из 10
/content/data/phishing/94.jpg: Уверенность фишинга 6.4 из 10
/content/data/phishing/106.jpg: Уверенность фишинга 3.2 из 10
/content/data/phishing/119.jpg: Уверенность фишинга 2.5 из 10
/content/data/phishing/26.jpg: Уверенность фишинга 10.0 из 10
/content/data/phishing/56.jpg: Уверенность фишинга 4.6 из 10
/content/data/phishing/8.jpg: Уверенность фишинга 1.7 из 10
/content/data/phis

# LLaVA

In [9]:
import torch
import os
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq

In [10]:
model_name = "llava-hf/llava-1.5-7b-hf"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForVision2Seq.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [19]:
def classify_image(image_path, question="Это фишинговая атака?"):
    if question is None:
        question = (
            "Проанализируй это изображение. Определи, содержит ли оно признаки фишинга, "
            "такие как поддельные страницы входа, QR-коды для входа в соцсети, "
            "фейковые банковские и биржевые страницы."
        )

    image = Image.open(image_path).convert("RGB")
    prompt = f"{question}"
    inputs = processor(images=[image], text=[question], return_tensors="pt")

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50)

    response = processor.batch_decode(output, skip_special_tokens=True)[0]
    return question, response

In [21]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_dir):
        self.dataset_dir = dataset_dir
        self.images = []

        for dirpath, _, filenames in os.walk(dataset_dir):
            for filename in filenames:
                img_path = os.path.join(dirpath, filename)
                self.images.append(img_path)  # Теперь храним только пути, без меток

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx]  # Возвращаем только путь к изображению


In [23]:
dataset_path = "/content/data"
dataset = ImageDataset(dataset_path)

results = {}
for image_path in dataset:

    try:
        image = Image.open(image_path).convert("RGB")
        prompt, response = classify_image(image_path)
        results[image_path] = {"prompt": prompt, "response": response}
        print(f"{image_path} | Промпт: {prompt} | Ответ: {response}")
    except Exception as e:
        print(f"Ошибка обработки изображения {image_path}: {e}")


Ошибка обработки изображения /content/data/normal/78.jpg: Image features and image tokens do not match: tokens: 0, features 576


KeyboardInterrupt: 